# Data Collection

Here I will take information from RISD Museum Collection API.  
Documentation can be found here: https://risdmuseum.org/art-design/projects-publications/articles/risd-museum-collection-api

## Import data

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import nltk

import time

import pickle

This API allow loading maximum 25 items at a time.  
There are about 3900 works with 'painting' search term in their collection. I'll run it through about 156 times.

In [9]:
url = "https://risdmuseum.org/api/v1/collection" #RISD Museum collection

db = []

for i in range(156):
    resp = requests.get(url, {'search_api_fulltext': 'painting', 
                'items_per_page': 25, 
                'page': i})
    db.extend(resp.json())
    time.sleep(np.random.randint(10, 3600))

### Downloading Images

In [ ]:
df = pd.DataFrame(db)

In [ ]:
image_id_set = df[['id', 'images']]

In [ ]:
image_id_set['image'] = [x[0]  if x else '' for x in image_id_set['images']]

In [ ]:
# downloading images
def download_img(df): 
    url = str(df['image'])
    if url.startswith('http'):
        try:
            resp = requests.get(url).content
        except:
            df['fname'] = 'error'
            return df

        fpath = f"data/IMAGES/RISD/{df['id']}.jpg"
        with open(fpath, 'wb') as fp:
            fp.write(resp)

        img = Image.open(fpath)
        img.thumbnail((500, 500))
        img.save(fpath)
        df['fname'] = fpath
    else:
        df['fname'] = None
    
    pause = np.random.randint(1, 60)
    print(f'pausing for {pause} secs ...')
    time.sleep(pause) # buffer
    
    return df

In [ ]:
for i in range(len(image_id_set)):
    image_id_set.iloc[i] = download_img(image_id_set.iloc[i])

### Exporting

In [ ]:
full_df = df.drop_duplicates(subset = 'id')
filename = 'pickles/raw_data_RISD.pkl'
full_df.to_pickle(filename)